In [ ]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.metrics import log_loss
from keras import models
from keras import layers
from keras import optimizers
import random

In [ ]:
#Preprocess data for LSTM neural net
def LSTM_preprocess(text,author):
    tk = Tokenizer(lower = True,num_words=5000)
    tk.fit_on_texts(text)
    X_seq = tk.texts_to_sequences(text)
    X = pad_sequences(X_seq,255)
    target = pd.get_dummies(author)
    return X, target

In [ ]:
#Instantiate neural net
def instantiate_LSTM(num_words = 5000, max_sequence_len=255, embed_vec_len=32):
    lstm_nn = models.Sequential()
    lstm_nn.add(layers.Embedding(num_words, embed_vec_len, input_length=max_sequence_len))
    lstm_nn.add(layers.SpatialDropout1D(0.2))
    lstm_nn.add(layers.Bidirectional(layers.LSTM(64)))
    lstm_nn.add(layers.Dense(3, activation='softmax'))
    lstm_nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return lstm_nn

In [ ]:
#Create ensemble of 20 LSTM neural nets, calculate weighted average of weights based on val loss, output agg pred
def ensemble_LSTM(n_estimators=20)
    val_loss =[]
    bag_nn_preds = []
    counter = 0
    while counter < n_estimators:
        print(counter+1)
        lstm_nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        sub = random.sample(range(len(X)),k=15000)
        val = [i for i in range(len(X)) if i not in sub]
        history = lstm_nn.fit(X[sub],target.iloc[sub,:],epochs=1,batch_size=256, validation_data=(X[val],target.iloc[val,:]))
        val_loss.append(1-history.history['val_loss'][-1])
        bag_nn_preds.append(lstm_nn.predict(X))
        counter += 1
    weights = np.array([i/sum(val_loss) for i in val_loss])
    arr = np.zeros(np.array(bag_nn_preds).shape)
    for i in range(len(weights)):
        arr[i]=weights[i]*np.array(bag_nn_preds)[i]
    bnp = np.mean(arr, axis=0)
    return bnp